![](top.png)

---

[Prodigy Documentation](https://spacy.apjan.co/docs/)

Basic NER with pre-trained spaCy models ([source](https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da))

In [1]:
import os 
import pickle
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

nlp = en_core_web_sm.load()
spec = {"tei":"http://www.tei-c.org/ns/1.0"}

doc = nlp(
    """The COST Action Distant Reading for European Literary History is issuing a Call for Applications for its third Training School, hosted by the Centre for Digital Humanities – Eötvös Loránd University in Budapest from September 23 to 25, and co-located with the DH_Budapest 2019 conference 
"""
)
displacy.render(doc, style="ent")

> Here is a [simple web app](https://explosion.ai/demos/displacy-ent) that you can use to quickly evaluate how useful an existing model would be for your current text or project.  For many languages, there are several possible [models](https://spacy.io/usage/models#languages).  This tool can be used to evaluate the results from different models.    

## Problem:
*This works very well for many 20th and 21st century texts.  But what about early modern English?*

In [2]:
doc = nlp(
    "ITEM because that the kings most deare Uncle, the king of Denmarke, Norway & Sweveland, as the same our soveraigne Lord the king of his intimation hath understood, considering the manifold & great losses, perils, hurts and damage which have late happened aswell to him and his, as to other foraines and strangers, and also friends and speciall subjects of our said soveraigne Lord the king of his Realme of England, by ye going in, entring & passage of such forain & strange persons into his realme of Norwey & other dominions, streits, territories, jurisdictions & places subdued and subject to him, specially into his Iles of Fynmarke, and elswhere, aswell in their persons as their things and goods"
)
displacy.render(doc, style="ent")

## Introduction to ✨Prodigy

Prodigy is an annotation tool for active machine teaching.  Prodigy makes it possible to quickly experiment and improve models with relatively little data, time and typically only one annotator.  Prodigy will sort a model's results by certainty, and ask for human input where the results are most ambiguous. Prodigy stores these annotations and uses them to update the model.  Machine teaching can be used to: 

- improve an existing model for a specific context or set of documents
- add a new entity or category to a text model 
- custom image categorization and object recognition

*While spaCy is a free and open-source library, Prodigy is a commercial product that requires a license. Researchers at degree-granting academic institutions can obtain a free research license [here](https://prodi.gy/forms/research-license).*

### In this example, our goal is to teach an existing English-language model to identify early modern place names.

There are several approaches that we could take to this problem.  Different approaches can lend better or worse results and experimentation is an essential part of any machine learning project. 

### How can we teach a statistical language model that Sweveland is a place?



**Manual annotation**

Some researchers may prefer to add seed annotations using the `ner.manual` recipe. For certain personalities, myself included, this is actually kind of fun and a good way to think about the text and the goals of your experiment.  We'll discuss how Prodigy can be used as a manual annotation tool near the end of this session.

If you only have a few seed terms you can also add them as an argument with `ner.teach en_core_web_lg ... --seeds "digital,humanities,utrecht,netherlands,spacy,workshop"`[(source)](https://support.prodi.gy/t/train-a-new-ner-entity-with-multi-word-tokens/227/3) 

In our current case, we're going to provide as many examples of place names as possible.  Note that with spaCy, we are working with place names as [patterns](https://spacy.apjan.co/docs/#match-patterns) so the model can learn that place names often have several parts.  

**Seed term patterns** 

We're going to give the model as many examples of historic place names as we can to get the learning process started. To do this, I have chosen to mine a text that has already been markedup in TEI with early modern place names and is available from the Perseus Project.  We're going to use Richard Hakluyt's [*The Principal Navigations, Voyages, Traffiques, and Discoveries of the English Nation* (1599)](http://www.perseus.tufts.edu/hopper/text?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D1).  I have chosen an English-language example because we'll be annotating the text in this workshop and English is a working language of DH2019.  However, you can work with texts in [any language](https://spacy.io/usage/adding-languages) even those that do not have an existing spaCy language model ([colonial Zapotec](https://ticha.haverford.edu/) for example!).  




**New category or existing?** 

We will need to choose whether we want to add a new entity to the model (let's call it "PLACE") or improve the existing "GPE" (Geopolitical entity, i.e. countries, cities, states) for our historic places.  Either one works.  If we improve an existing entity, we retain the existing training and examples. A new entity makes it easier to distinguish between what we have taught the model and previous training.  This can also address potential bias in previous training data.  For that reason, we'll add "PLACE" as a new category.     

### Download the TEI files from Persius 
- We're going to extract a list of all the place names from the text to create a patterns JSONL file.
- We'll also extract the raw text to create a set of training documents. 

We are going to download the table of contents and create a list of the 937 segments of the document. We will then get each page, remove the place names (`<name type="place">Utrect</name>`) and add them to a places list.

In [3]:
from urllib.request import urlopen
from lxml import etree

def tei_loader(url):
    tei = urlopen(url).read()
    return etree.XML(tei)

table_of_contents_url = "http://www.perseus.tufts.edu/hopper/xmltoc?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D1"
table_of_contents_xml = tei_loader(table_of_contents_url)

if not os.path.exists('refs.pickle'):
    chunks = table_of_contents_xml.xpath("//chunk[@ref]")
    refs = [chunk.get('ref') for chunk in chunks] 
    # an example ref 'Perseus%3Atext%3A1999.03.0070%3Anarrative%3D6'


    places = []

    for ref in refs:

        url = 'http://www.perseus.tufts.edu/hopper/xmlchunk?doc=' + ref
        try:
            tei = tei_loader(url)

            #get all <name type='place'> tags
            for place in tei.findall(".//name[@type='place']", namespaces=spec):
                places.append(place.text.replace('\n',''))
        except Exception as e:
            print(e)
            
    pickle.dump(places, open('places.pickle', 'wb'))
    pickle.dump(refs, open('refs.pickle', 'wb'))

else:
    places = pickle.load(open('places.pickle', 'rb'))
    refs = pickle.load(open('refs.pickle', 'rb'))
    print('pickles loaded')


pickles loaded


In [4]:
print('number of documents: ', len(refs))
print('number of places found: ', len(set(places)))
places[9]

number of documents:  937
number of places found:  2279


'York'

### Create a patterns.jsonl file with seed terms.  
These terms provide examples that the model can use to learn the new category. It is good to use as many terms as are practical; preferably several hundred. Further examples of patterns files can be found [here](https://github.com/explosion/prodigy-recipes/tree/master/example-patterns). There is a very helpful tool for creating patterns that are relevant to your projects and texts [here](https://explosion.ai/demos/matcher).


In [5]:
import json
new_label = 'PLACE'

if not os.path.exists('patterns.jsonl'):
    with open('patterns.jsonl','w') as f:
        for place in set(places):   # A set is used here to remove duplicate place names
            
            row = {}
                
            row['label'] = new_label
            row['pattern'] = []
            for token in place.split():
                pattern = {}
                pattern['lower'] = token.lower()
                row['pattern'].append(pattern)
        
            f.write(str(json.dumps(row) +'\n')) 
            
        '''Polite intervention:  
        Sweveland is not in our list of historic and beautiful places, but is essential to the narrative of this notebook.
        To correct this error, the line below has been added, with appologies to our colleagues from Sweveland'''
        row = {"label": "PLACE", "pattern": [{"lower": "sweveland"}]}
        f.write(json.dumps(row))

In [6]:
with open('patterns.jsonl','r') as f:
    print(f.read()[:185])

{"label": "PLACE", "pattern": [{"lower": "wardhouse"}]}
{"label": "PLACE", "pattern": [{"lower": "sea"}, {"lower": "southwest"}]}
{"label": "PLACE", "pattern": [{"lower": "silauria"}]}



In [7]:
#Now to extract the full text
percent_of_texts_to_process = 0.05 
'''We are only using a tiny portion of the text for annotation.  
Unlike many machine learning tasks, in this case, more text data for annotations does not necessarily
improve learning. More text will dramatically increase Prodigy's memory usage, so keep that in mind.'''

end_index = int(len(refs) * percent_of_texts_to_process)

if os.path.exists("principal_navigations.txt"):
    print(
        "The file already exists. This process takes several minutes. To re-run, change process_anew to True. You can also adjust the percentage of the corpus to be gathered."
    )
    
process_anew = False
if process_anew:
    txts = []
    for ref in refs[:end_index]:

        url = 'http://www.perseus.tufts.edu/hopper/xmlchunk?doc=' + ref
        try:
            tei = tei_loader(url)

            new_txt = []
            for body in tei.iter('body'):
                new_txt.append(''.join(body.itertext()).strip().replace('\n',''))
                txts.append(''.join(new_txt))

        except Exception as e:
            print(e)
            

    full_text = [
        txt.replace("        ", " ").replace("   ", " ").replace("  ", " ")
        for txt in txts
    ]
    
    with open('principal_navigations.txt','w') as f:
        f.write(str(full_text))

The file already exists. This process takes several minutes. To re-run, change process_anew to True. You can also adjust the percentage of the corpus to be gathered.


In [8]:
with open('principal_navigations.txt','r') as f: 
    print(f.read()[:600])
    

['A branch of a Statute made in the eight yeere of Henry the sixt, for the trade to Norwey, Sweveland, Den marke, and Fynmarke. ITEM because that the kings most deare Uncle, the kingof Denmarke, Norway & Sweveland, as the same oursoveraigne Lord the king of his intimation hath understood, considering the manifold & great losses, perils,hurts and damage which have late happened aswell tohim and his, as to other foraines and strangers, and alsofriends and speciall subjects of our said soveraigne Lordthe king of his Realme of England, by ye going in,entring & passage of such forain & strange pers


## With patterns and text files created, we can now work with ✨Prodigy!

The `dataset` command will create a database table to save your annotations. The default is sqlite, but you can connect to [MySQL or postgres](https://spacy.apjan.co/docs/#database-setup)

In [9]:
!prodigy dataset DH-Budapest "A dataset for British historic places" --author Andy


  ✨  ERROR: 'DH-Budapest' already exists in database SQLite.



If you'd like to delete a dataset:   

`prodigy drop historic_places`



## A. Plain text to TEI. 


To start the annotation application for a named entity recognition task, we use the `ner.teach` recipe. Similar [built-in recipes](https://spacy.apjan.co/docs/#built-in-recipes) are available for: 
- text categorization (textcat.teach) 
- part of speech tagging (pos.teach)  
- vectors & terminology (terms.teach)
- computer vision (image.manual)

In the command below, we use the `ner.teach` recipe to annotate the `historic_places` dataset, using the `en_core_web_sm` model to add the new entity `PLACE` after loading the patterns in `patterns.jsonl`.  

*Click on the stop button to intterupt the kernel when you're done.*

In [10]:
!prodigy ner.teach DH-Budapest en_core_web_sm principal_navigations.txt --label PLACE --patterns patterns.jsonl

Using 1 labels: PLACE

  ✨  Starting the web server at http://localhost:8080 ...
  Open the app in your browser and start annotating!

Task queue depth is 1
^C

Saved 61 annotations to database SQLite
Dataset: DH-Budapest
Session ID: 2019-10-07_12-01-17



Once you have completed adding annotations, the next step is to train the model.  

In the command below, we use `ner.batch-train` to use the annotations in the `historic_places` dataset to train the `en_core_web_sm` model on the new entity `PLACE`.  We then save the updated model as `new_model`.

In [11]:
!prodigy ner.batch-train DH-Budapest en_core_web_sm --label PLACE --output new_model 

Using 1 labels: PLACE

Loaded model en_core_web_sm
Using 50% of accept/reject examples (10) for evaluation
Using 100% of remaining examples (10) for training
Dropout: 0.2  Batch size: 4  Iterations: 10  


BEFORE      0.000            
Correct     0  
Incorrect   13
Entities    56               
Unknown     0                

#            LOSS         RIGHT        WRONG        ENTS         SKIP         ACCURACY  
01           620.858      0            13           232          0            0.000     
02           661.659      1            12           198          0            0.077     
03           656.434      1            12           112          0            0.077     
04           631.189      0            13           72           0            0.000     
05           594.486      0            13           47           0            0.000     
06           567.271      0            13           30           0            0.000     
07           588.344      1            12        

If you think the model would benefit from more training you can run this process again to load and update `new_model`.  You can also add the `--n-iter` argument to specify the number of iterations.

In [13]:
!prodigy ner.batch-train DH-Budapest new_model --output new_model --label PLACE --n-iter 120

Using 1 labels: PLACE

Loaded model new_model
Using 50% of accept/reject examples (10) for evaluation
Using 100% of remaining examples (10) for training
Dropout: 0.2  Batch size: 4  Iterations: 120  


BEFORE      0.231            
Correct     3  
Incorrect   10
Entities    53               
Unknown     48               

#            LOSS         RIGHT        WRONG        ENTS         SKIP         ACCURACY  
01           273.957      2            11           21           0            0.154     
02           175.063      2            11           15           0            0.154     
03           113.121      2            11           14           0            0.154     
04           120.217      4            9            15           0            0.308     
05           145.320      4            9            17           0            0.308     
06           139.278      3            10           10           0            0.231     
07           132.138      3            10           9

Would our model improve with more data?

In [12]:
!prodigy ner.train-curve DH-Budapest new_model --label PLACE --n-iter 10 --eval-split 0.2 --dropout 0.2  --n-samples 4 

Using 1 labels: PLACE

Starting with model new_model
Dropout: 0.2  Batch size: 32  Iterations: 10  Samples: 4

%            RIGHT        WRONG        ACCURACY  
25%          1            5            0.17         +0.17         
50%          0            6            0.00         -0.17        
75%          0            6            0.00         +0.00        
100%         0            6            0.00         +0.00        


To load and see the results of our new model 

In [14]:
import spacy

nlp = spacy.load("new_model")
doc = nlp(
    "ITEM because that the kings most deare Uncle, the king of Denmarke, Norway & Sweveland, as the same our soveraigne Lord the king of his intimation hath understood, considering the manifold & great losses, perils, hurts and damage which have late happened aswell to him and his, as to other foraines and strangers, and also friends and speciall subjects of our said soveraigne Lord the king of his Realme of England, by ye going in, entring & passage of such forain & strange persons into his realme of Norwey & other dominions, streits, territories, jurisdictions & places subdued and subject to him, specially into his Iles of Fynmarke, and elswhere, aswell in their persons as their things and goods"
)
displacy.render(doc, style="ent")

That's not too bad.  What about a text that the model has never seen before? Let's try Çelebi Evliya's [Narrative of travels in Europe, Asia, and Africa](https://archive.org/details/narrativeoftrave01evli/page/n4)

In [15]:
import spacy

nlp = spacy.load("new_model")
doc = nlp(
    """The army marched from Konia to Kaiseria (Caesarea), and thence to Sivas, where the feast of the Korbân (sacrifice) was celebrated. Here Mustafâ Pâshâ, the emperor's favourite, was promoted to the rank of second vezir, and called into the divân. The army then continued its march to Erzerum. Besides tiie guns provided by the commander-in-chief, there were forty large guns dragged by two thousand pairs of buftaloes. The army entered the castle of Kazmaghan, and halted under the walls of Eriviin in the year 1044 (1634).  
"""
)

counter = 0
for ent in doc.ents:
    if ent.text in places:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
        counter += 1

print(f"{counter} of the place entities were in the training data")
displacy.render(doc, style="ent")

0 of the place entities were in the training data


Mustafâ Pâshâ is a person, but the model has done as passable job. Let's use our model to automatically identify historical place names and produce a markedup TEI document. 

In [16]:
text = """The army marched from Konia to Kaiseria (Caesarea), and thence to Sivas, where the feast of the Korbân (sacrifice) was celebrated. Here Mustafâ Pâshâ, the emperor's favourite, was promoted to the rank of second vezir, and called into the divân. The army then continued its march to Erzerum. Besides tiie guns provided by the commander-in-chief, there were forty large guns dragged by two thousand pairs of buftaloes. The army entered the castle of Kazmaghan, and halted under the walls of Eriviin in the year 1044 (1634)."""  

import spacy
nlp = spacy.load('new_model')
doc = nlp(text)
text_list = [i.text for i in doc]

for token in doc:
    if token.ent_type_ == 'PLACE': 
        text_list[token.i] = '<name type="place">' + text_list[token.i] + '</name>'

punct = ['.',"'",',',')',':',';']

text=''
for i, token in enumerate(text_list):
    try:
        if text_list[i+1] in punct:
            text += token

        else: 
            text += token + ' '
        
    except IndexError:
        pass
   
    
text.replace('\n','').replace('( ','(')

'The army marched from <name type="place">Konia</name> to <name type="place">Kaiseria</name> (<name type="place">Caesarea</name>), and thence to <name type="place">Sivas</name>, where the feast of the <name type="place">Korbân</name> (sacrifice) was celebrated. Here Mustafâ Pâshâ, the emperor \'s favourite, was promoted to the rank of second vezir, and called into the divân. The army then continued its march to Erzerum. Besides tiie guns provided by the commander - in - chief, there were forty large guns dragged by two thousand pairs of buftaloes. The army entered the castle of <name type="place">Kazmaghan</name>, and halted under the walls of Eriviin in the year 1044 (1634)'

In [17]:
# save text as tei 
filename = 'my_tei.xml'
language = 'en'

tei_string = f"""
<TEI.2>
  <text lang="{language}">
    <body>
        <p>
            {text}
        </p>
    </body>
  </text>
</TEI.2>
"""
doc = etree.fromstring(tei_string)
tree = etree.ElementTree(doc)
tree.write(f'{filename}', pretty_print=True, xml_declaration=False,   encoding="utf-8")


In [18]:
with open('my_tei.xml','r') as f:
    print(f.read())

<TEI.2>
  <text lang="en">
    <body>
        <p>
            The army marched from <name type="place">Konia</name> to <name type="place">Kaiseria</name> ( <name type="place">Caesarea</name>), and thence to <name type="place">Sivas</name>, where the feast of the <name type="place">Korbân</name> ( sacrifice) was celebrated. Here Mustafâ Pâshâ, the emperor 's favourite, was promoted to the rank of second vezir, and called into the divân. The army then continued its march to Erzerum. Besides tiie guns provided by the commander - in - chief, there were forty large guns dragged by two thousand pairs of buftaloes. The army entered the castle of <name type="place">Kazmaghan</name>, and halted under the walls of Eriviin in the year 1044 ( 1634)
        </p>
    </body>
  </text>
</TEI.2>



### B. TEI to TEI


In [ ]:
!prodigy dataset tei_to_tei "A dataset for British historic places for the TEI to TEI section" --author Andy

The cell below uses David Lassner's [standoff converter](https://github.com/millawell/standoffconverter).  The tree_to_standoff() function returns a tuple with the plain text at index 0, a list of dictionaries for each xml tag at 1. Each tag contains the 'begin' and 'end' index in the text, the 'tag' as well as a dictionary of attributes ['attrib'] and ['depth'] 

```json
[{'begin': 0, 'tag': 'TEI.2', 'attrib': {}, 'depth': 0, 'end': 51499},  ... ]
```

In [ ]:
import json
import srsly   #https://pypi.org/project/srsly/
import standoffconverter

url = 'http://www.perseus.tufts.edu/hopper/xmlchunk?doc=' + refs[2]
print(url)

tei = urlopen(url).read()
tei = etree.XML(tei)

jsonl = {}
standoff = standoffconverter.tree_to_standoff(tei)
jsonl["text"] = standoff[0]
with open('my_tei.jsonl', 'w') as outfile:  
    json.dump(jsonl, outfile)

show = False 
if show:
    with open('my_tei.jsonl','r') as fr:
            jsonl = fr.read()
            print(json.loads(jsonl)['text'])
    

In [ ]:
!prodigy ner.teach tei_to_tei en_core_web_sm my_tei.jsonl --patterns patterns.jsonl

In [ ]:
import json
import standoffconverter

url = 'http://www.perseus.tufts.edu/hopper/xmlchunk?doc=' + refs[1]
print(url)

tei = urlopen(url).read()

tree = etree.XML(tei)

standoff = standoffconverter.Standoff()

standoff.from_lxml_tree(tree)

jsonl = {}

text = standoff.plain
nlp = spacy.load('new_model')
doc = nlp(text)

for ent in doc.ents:

    new_tag = {}
    new_tag['begin'] = ent.start_char
    new_tag['end'] = ent.end_char
    new_tag['tag'] = ent.label_  # change PERSON to a valid TEI label
    new_tag['attrib'] = {}
    new_tag['depth'] = 0
    standoff.standoffs.append(new_tag)

    
new = standoff.to_xml()
new

## Prodigy as a manual annotation tool 
text > tei manual markup

In [ ]:
text = """The army marched from Konia to Kaiseria (Caesarea), and thence to Sivas, where the feast of the Korbân (sacrifice) was celebrated. Here Mustafâ Pâshâ, the emperor's favourite, was promoted to the rank of second vezir, and called into the divân. The army then continued its march to Erzerum. Besides tiie guns provided by the commander-in-chief, there were forty large guns dragged by two thousand pairs of buftaloes. The army entered the castle of Kazmaghan, and halted under the walls of Eriviin in the year 1044 (1634)."""  
with open('text2tei.txt','w') as f:
    f.write(text)

In [ ]:
!prodigy dataset text2tei "A dataset for British historic places with text input" --author Andy

In [ ]:
!prodigy ner.manual text2tei en_core_web_sm text2tei.txt

In [ ]:
!prodigy db-out text2tei .

In [ ]:
with open('text2tei.jsonl','r') as f:
    jsonl = json.loads(f.read())
    print(jsonl['spans'])

In [ ]:
import json 

def jsonl_to_xml(jsonl):    
    jsonl = json.loads(jsonl)
    text = jsonl['text']
    offset = 0
    
    for span in jsonl['spans']:

        new_text = f"<{span['label']}>" + text[span["start"] +offset : span["end"] + offset] +  f"</{span['label']}>" 
        text = text[:span["start"] + offset] + new_text + text[span["end"] + offset:]
        offset += len(new_text) - (span["end"] - span["start"])
    
    return text
        

with open('text2tei.jsonl','r') as f:
    jsonl = f.read()
    xml = jsonl_to_xml(jsonl)
    print(xml)

In [ ]:
# save text as tei 
filename = 'annotated_tei.xml'
language = 'en'

tei_string = f"""
<TEI.2>
  <text lang="{language}">
    <body>
        <p>
            {xml}
        </p>
    </body>
  </text>
</TEI.2>
"""
doc = etree.fromstring(tei_string)
tree = etree.ElementTree(doc)
tree.write(f'{filename}', pretty_print=True, xml_declaration=False,   encoding="utf-8")

with open(filename,'r') as f:
    print(f.read())

For complex annotation tasks, see [brat](https://brat.nlplab.org/index.html), ([demo](http://weaver.nlplab.org/~brat/demo/latest/#/))

# Thank you for your attention!  How might you use spaCy and Prodigy in your research? 